In [36]:
import numpy as np
import openai
import os
import pandas as pd
from stanza.models.ner import scorer
from stanza.utils.confusion import format_confusion


In [31]:
# Read in manually annotated instagram captions and model's predictions
file_loc = '/manual_annotated_feb_ig_subset.tsv'
file_path = os.getcwd() + file_loc
model_output = pd.read_table(file_path)
model_output.columns=["Token", "Assigned", "Predicted"]
model_output.reset_index()
model_output.info()

,Token,Assigned,Predicted
0,What,O,O
1,an,O,O
2,awesome,O,O
3,day,O,O
4,with,O,O


In [30]:
# Print evaluation results
preds = [[]]
gold_tags = [[]]
for i in range(len(model_output)):
    preds[0].append(model_output.iloc[i]['Predicted'])
    gold_tags[0].append(model_output.iloc[i]['Assigned'])

_, _, score = scorer.score_by_entity(preds, gold_tags)
_, _, _, confusion = scorer.score_by_token(preds, gold_tags)

print(format_confusion(confusion))


2023-03-13 17:26:11 INFO: Score by entity:
Prec.	Rec.	F1
76.19	62.34	68.57
2023-03-13 17:26:11 INFO: Score by token:
Prec.	Rec.	F1
79.05	64.84	71.24


     t/p      O B-DAT E-DAT I-DAT S-DAT B-ITM E-ITM I-ITM S-ITM B-LOC E-LOC I-LOC S-LOC B-ORG E-ORG I-ORG S-ORG S-UNT S-WEI
        O  1115     0     0     0     1     2     0     0     1     0     1     0     1     2     2     1     0     0     4
    B-DAT     1     4     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0
    E-DAT     3     0     3     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0
    I-DAT     0     0     1     1     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0
    S-DAT     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0
    B-ITM     2     0     0     0     0     3     0     0     1     0     0     0     0     0     0     0     0     0     0
    E-ITM     3     0     0     0     0     0     3     0     0     0     0     0     0     0     0     0     0     0     0
    I-IT

In [34]:
# Feed all the sentences into GPT-3 and let it predict which ones are Orgs, location, etc.
post_delimiters = []
print(model_output['Token'][0:50])

0          What
1            an
2       awesome
3           day
4          with
5       amazing
6        people
7           and
8       stellar
9         dives
10            ,
11          and
12           we
13         left
14          the
15        ocean
16       better
17         than
18           we
19        found
20           it
21            !
22            🌎
23           In
24        total
25           we
26     gathered
27        about
28           50
29          lbs
30           of
31      debris-
32          the
33      biggest
34       weight
35       factor
36        being
37            a
38       anchor
39          and
40        chain
41         that
42          was
43         left
44       behind
45            ,
46          but
47          the
48    remaining
49           10
Name: Token, dtype: object


In [37]:
fname = "api_key.txt"
with open(fname, "r") as f:
    OPENAI_KEY = f.readline().strip()

openai.api_key = OPENAI_KEY

In [38]:
def get_prompt(row):
    prompt = f"Please extract the organization, location, date, type of trash, and weight of trash from this post: {row['text']}"

response = openai.Completion.create(
                engine="text-davinci-003",
                prompt= "Who is your daddy?",
                max_tokens=75,
                temperature=0.7
            )
print(response['choices'][0]['text'])



I don't understand the question.
